# Задание
Выполните следующие задания:

> 1. Лёгкий уровень (Задание не обязательно к выполнению)
Запишите видео с собой (допустим, держащим лист с надписью «1Т») из примера ноутбука занятия. Вставьте скриншот себя в ваш ноутбук с кодом (вы должны быть обведены в зеленый квадрат, детектированы нейронной сетью как в скринкастах), на котором будут видны сопровождающие надписи вокруг рамки видео: «Захват изображения..."» и «Чтобы завершить...».

Примечание: Если вам некомфортно снимать себя и прикладывать скриншот к ноутбуку, можете использовать какое-либо видео из интернета.

> 2. PRO уровень (Задание не обязательно к выполнению)
Доработав код захвата видео с камеры, сделайте запись видео с собой сегментированным, выводящим рукой надпись «1Т». Приложите ту часть кода на Python, которая ответственна за создание файла видео. Так же, как и в легком уровне, приложите скриншот сегментированного человека (это можете быть вы или какое-либо видео с человеком).

Примечание: Если вам некомфортно снимать себя и прикладывать скриншот к ноутбуку, можете использовать какое-либо видео из интернета.

In [1]:
!pip install ultralytics

<div class="alert alert-info">

1. Лёгкий уровень (Задание не обязательно к выполнению) Запишите видео с собой (допустим, держащим лист с надписью «1Т») из примера ноутбука занятия. Вставьте скриншот себя в ваш ноутбук с кодом (вы должны быть обведены в зеленый квадрат, детектированы нейронной сетью как в скринкастах), на котором будут видны сопровождающие надписи вокруг рамки видео: «Захват изображения..."» и «Чтобы завершить...».

In [2]:
from IPython.display import Javascript
from IPython.display import Image as Image2
from IPython import display
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import torch
from PIL import Image as Image1
import gc

In [3]:
detector = torch.hub.load('ultralytics/yolov5', 'custom', path = 'yolov5m.pt') #'yolov5s.pt'
print('#'*60)
print(f"Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'}), OpenCV: {cv2.__version__}")

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
WARNING ⚠️ 'ultralytics.yolo.v8' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.models.yolo' instead.
WARNING ⚠️ 'ultralytics.yolo.utils' is deprecated since '8.0.136' and will be removed in '8.1.0'. Please use 'ultralytics.utils' instead.
Note this warning may be related to loading older models. You can update your model to current structure with:
    import torch
    ckpt = torch.load("model.pt")  # applies to both official and custom models
    torch.save(ckpt, "updated-model.pt")

YOLOv5 🚀 2023-7-30 Python-3.10.6 torch-2.0.1+cu118 CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients
Adding AutoShape... 


############################################################
Using torch 2.0.1+cu118 (CPU), OpenCV: 4.7.0


In [4]:
coco_labels = '''person
bicycle
car
motorcycle
airplane
bus
train
truck
boat
traffic light
fire hydrant
street sign
stop sign
parking meter
bench
bird
cat
dog
horse
sheep
cow
elephant
bear
zebra
giraffe
hat
backpack
umbrella
shoe
eye glasses
handbag
tie
suitcase
frisbee
skis
snowboard
sports ball
kite
baseball bat
baseball glove
skateboard
surfboard
tennis racket
bottle
plate
wine glass
cup
fork
knife
spoon
bowl
banana
apple
sandwich
orange
broccoli
carrot
hot dog
pizza
donut
cake
chair
couch
potted plant
bed
mirror
dining table
window
desk
toilet
door
tv
laptop
mouse
remote
keyboard
cell phone
microwave
oven
toaster
sink
refrigerator
blender
book
clock
vase
scissors
teddy bear
hair drier
toothbrush
hair brush'''.split()

In [5]:
def js_to_image(js_reply):
    """
    Params:
        js_reply: JavaScript object containing image from webcam
    Returns:
        img: OpenCV BGR image
    """
    # декодируем изображение из формата base64
    image_bytes = b64decode(js_reply.split(',')[1])
    # конвертируем байты в numpy array
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    # декодируем numpy array в изображение OpenCV BGR image
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

# функция для преобразования изображения с прямоугольником OpenCV Rectangle bounding box image в байтовую строку base64 чтобы вывести в видеопоток
def bbox_to_bytes(bbox_array):
    """
    Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
    Returns:
          bytes: Base64 image byte string
    """
    # конвертируем массив в изображение PIL image
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    # преобразуем bbox в формат png для возврата из функции с помощью return
    bbox_PIL.save(iobuf, format='png')
    # форматируем возвращаемую с помощью return строку
    bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
    return bbox_bytes

In [6]:
# Используем код на JavaScript для захвата потока с нашей вебкамеры и передачи данных на облачный компьютер - надо запускать также для следующего раздела
def video_stream_js():
    js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'Чтобы завершить, щёлкните мышкой на данном видео</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')
    
    display.display(js)

def video_frame(label, bbox):
    data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
    return data

In [7]:
# получаем прямоугольники(баундбоксы) - в кадре должен быть человек
# очищаем память
gc.collect()

# начинаем захват видео с камеры
video_stream_js()

# Создаём надпись
label_html = 'Захват изображения...'

# инициализируем пустые прямоугольники выделения
bbox = ''
count = 0
while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # конвертируем поток, передаваемый через JS, в изображение OpenCV Image
    img = js_to_image(js_reply["img"])

    # создаём пустой массив, в котором будет отображаться детекция объектов
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)

    # переводим изображение в серое для детекции
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # детектирование
    cur = detector(img) #img
    yolo_preds = cur.xyxy[0].cpu().numpy()

    # получение коодинат прямоугольников
    boxes = yolo_preds[:, :4]  # xmin, ymin, xmax, ymax
    confs = yolo_preds[:, 4]
    labels = yolo_preds[:, 5]

    fst_box_flag = 1

    # получаем конкретные прямоугольники
    for box, conf, label in zip(boxes, confs, labels):
        if coco_labels[int(label)] == 'person' and fst_box_flag == 1: #
            y1, y2, x1, x2 = int(box[1]), int(box[3]), int(box[0]), int(box[2])
            bbox_array = cv2.rectangle(bbox_array,(x1,y1),(x2,y2),(0,255,0),2)
            fst_box_flag = 0

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255

    bbox_bytes = bbox_to_bytes(bbox_array) #bbox_array
    # записываем в конкретные массивы изображения с прямоугольниками
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

**Выводим фото**
![Снимок](./снимок.jpg)

<div class="alert alert-info">

2. PRO уровень (Задание не обязательно к выполнению) Доработав код захвата видео с камеры, сделайте запись видео с собой сегментированным, выводящим рукой надпись «1Т». Приложите ту часть кода на Python, которая ответственна за создание файла видео. Так же, как и в легком уровне, приложите скриншот сегментированного человека (это можете быть вы или какое-либо видео с человеком).



<div style='border:solid blue 2px; padding: 20px'>
Прошу сделать разбор PRO части на вебинаре
    (Тема крутая, но недопонимание огромное)

In [8]:
# Функция для добавления надписи на изображение
def add_text(image):
    font = cv2.FONT_HERSHEY_SIMPLEX
    text = '1Т'
    org = (50, 50)  # Координаты начала надписи
    font_scale = 2
    color = (255, 255, 255)  # Цвет текста (белый)
    thickness = 2
    image = cv2.putText(image, text, org, font, font_scale, color, thickness, cv2.LINE_AA)
    return image

# Захват видео с камеры
cap = cv2.VideoCapture(0)

# Задаем формат видео и FPS
fourcc = cv2.VideoWriter_fourcc(*'XVID')
fps = cap.get(cv2.CAP_PROP_FPS)

# Указываем имя файла видео и параметры записи
out = cv2.VideoWriter('segmented_video.avi', fourcc, fps, (640, 480))

# Загружаем модель сегментации
#net = cv2.dnn.readNetFromCaffe('path/to/model.prototxt', 'path/to/model.caffemodel')

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Обработка кадра

    # Размещение надписи "1Т"
    frame = add_text(frame)

    # Записываем кадр в файл видео
    out.write(frame)

    cv2.imshow('Segmented Video', frame)  # Отображение видео на экране

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()